### Import Packages

In [ ]:
import numpy as np
import altair as alt
import pandas as pd
import warnings
# import vegafusion

warnings.filterwarnings('ignore')

# !pip3 install numpy
# !pip3 install -U numexpr bottleneck
# !pip3 install "numpy<2.0" 
# !pip3 install numpy==1.26.4
# !pip3 uninstall numpy -y

# !pip3 show numpy


### EDA Exploratory Data Analysis

In [ ]:
df = pd.read_csv("Boonsong_Lekagul_waterways_readings.csv")

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df["location"].unique()

In [ ]:
df["measure"].unique()

In [ ]:
# Find location with highest records
df['location'].value_counts()

In [ ]:
# Find location with most repeated measurements
df.groupby('location')['measure'].nunique().sort_values(ascending=False)


#### Based on the above dataset, below are some findings.

1. No missing values.
2. There are total of 136824 records in dataset.
3. The most repeated location is Boonsri with 31314 records.
4. Busarakhan is the location with most variety of measurements

### Feature Engineering

#### Convert 'sample date' to datetime in order to extract additional time based features which will help us to achieve below findings.

1. Does water quality vary by year or month?
2. Are there any seasonal patterns in measurements?
3. Is there a difference in readings between weekdays and weekends?

#### Lets start by creating some basic visuals and then we will progress as we go on.

In [ ]:
# Convert 'sample date' to datetime and create new columns in one step
df['sample date'] = pd.to_datetime(df['sample date'])
df['year'] = df['sample date'].dt.year
df['month'] = df['sample date'].dt.month
df['year_month'] = df['sample date'].dt.to_period('M').astype(str)
df['week_day'] = df['sample date'].dt.dayofweek
df.info()


### Visuals

In [ ]:
# !pip3 install "vegafusion==1.5.0" "vegafusion-python-embed==1.5.0"
# !pip3 install "vl-convert-python>=1.6.0"
# !pip3 install "pyarrow==10.0.1"
alt.data_transformers.enable('default')
alt.data_transformers.disable_max_rows()

# !pip3 show vegafusion
# !pip3 show vegafusion-python-embed
# !pip3 show pyarrow
# import vegafusion

# Basic bar chart: Count of records by location
bar_chart = alt.Chart(df).mark_bar().encode(x=alt.X('location:N', title='Location'), y=alt.Y('count():Q', title='Count of Records'), color=alt.Color('count():Q', scale=alt.Scale(scheme='blues'), title='Count of Records'), tooltip=[alt.Tooltip('location:N', title='Location'), alt.Tooltip('count():Q', title='Count of Records')]).properties(title="Count of Records by Location with Color Intensity", width=700, height=400)
bar_chart



##### Key Insights

Bar chart

Boonsri & Chai are the 2 topmost locations with highest number of records, followed by Kannika and Sakda. Tansanee, Decha & Achara are the locations with least records.

In [ ]:
# Scatter plot: Value vs. Year colored by measure
scatter_chart_year = alt.Chart(df).mark_circle(size=60).encode(x='year:O', y='value:Q', color='measure:N', tooltip=['location', 'value', 'measure', 'year']).properties(title="Scatter Plot: Value vs. Year", width=700, height=400)
scatter_chart_year

In [ ]:
# Scatter plot: Value vs. Month colored by measure
scatter_chart_month = alt.Chart(df).mark_circle(size=60).encode(x='month:O', y='value:Q', color='measure:N', tooltip=['location', 'value', 'measure', 'year']).properties(title="Scatter Plot: Value vs. Month", width=700, height=400)
scatter_chart_month

##### Key Insights

Scatter charts

In the year-based scatter plot, there are noticeable spikes in 2003, particularly for the measure "Iron." Similarly, in the month-based scatter plot, significant spikes are observed in August, primarily for the measure "Iron," followed by "Total Coliforms."

In [ ]:
# Heatmap: Average value by location and month
heatmap = alt.Chart(df).mark_rect().encode(x='month:O', y='location:N', color='mean(value):Q', tooltip=['month', 'location', 'mean(value)']).properties(title="Heatmap: Average Value by Location and Month", width=700, height=400)
heatmap


In [ ]:
# Heatmap: Count of records by location and year
heatmap = alt.Chart(df).mark_rect().encode(x=alt.X('year:O', title='Year'), y=alt.Y('location:N', title='Location'), color=alt.Color('count():Q', scale=alt.Scale(scheme='viridis'), title='Count of Records'), tooltip=[alt.Tooltip('year:O', title='Year'), alt.Tooltip('location:N', title='Location'), alt.Tooltip('count():Q', title='Count of Records')]).properties(title="Heatmap: Count of Records by Location and Year", width=700, height=400)
heatmap


##### Key Insights

Month-Based
The location Tansansee consistently exhibits higher values of chemical components across all 12 months of the year. Kohsoom displays high intensities of chemical components specifically in August, while Achara shows notable intensities in January.

Year-Based
Data is missing for three locations between 1998 and 2008. From 2009 to 2016, chemical components are consistently higher across these locations. Additionally, Somchair, Kohsoom, and Busarakhan demonstrate similar trends in chemical components throughout the entire year range, spanning from 1998 to 2016.

In [ ]:
# Bar chart: Dropdown by location
selector = alt.selection_point(fields=['location'], bind=alt.binding_select(options=df['location'].unique().tolist()), name="Select")

bar_chart_location_year = alt.Chart(df).mark_bar().encode(x='year:O', y='mean(value):Q', color='location:N', tooltip=['year', 'mean(value):Q', 'location']).add_params(selector).transform_filter(selector).properties(title="Bar Chart: Average Values by Year and Location", width=800, height=400)
bar_chart_location_year


##### Key Insights

This visual enables us to explore the values across different years dynamically, based on the selected location from the dropdown menu.

•	Boonsri: Records show an upward trend as the years progress.
•	Kannika: Displays a notable spike in 2003.
•	Chai, Kohsoom, Somchair, Sakda, Busarakhan: All exhibit a similar spike in 2003.
•	Tansanee: Data remains almost consistent across the years.
•	Achara, Decha: Both show a significant spike in 2009.

In [ ]:
# Bar chart: Dropdown by location
selector = alt.selection_point(fields=['location'], bind=alt.binding_select(options=df['location'].unique().tolist()), name="Select")

bar_chart_location_measure = alt.Chart(df).mark_bar().encode(x='measure:N', y='mean(value):Q', color='measure:N', tooltip=['measure', 'mean(value):Q', 'location']).add_params(selector).transform_filter(selector).properties(title="Bar Chart: Average Values by Measure and Location", width=800, height=400)
bar_chart_location_measure


##### Key Insights

Kohsoom, Achara, Decha: Total coliforms have the highest values.
Other Locations (Boonsri, Busarakhan, Sakda, Somchair, Chai, Kannika): Total dissolved salts have the highest values, followed by bicarbonates. Aluminium is also observed to have relatively high values in these locations.



In [ ]:
# Define the selection for location (radio buttons)
location_selector = alt.selection_point(fields=['location'], bind=alt.binding_radio(options=df['location'].unique().tolist(), name="Location: "),)

measurement_selector = alt.selection_point(fields=['measure'], bind=alt.binding_select(options=['Total dissolved salts', 'Bicarbonates', 'Oxygen saturation', 'Total hardness', 'Zinc', 'Calcium', 'Iron'], name="Measurement: "), name="Select Measurement")

time_series_chart = alt.Chart(df).mark_line().encode(x=alt.X('year:O', title='Year'), y=alt.Y('value:Q', title='Measurement Value'), color=alt.Color('measure:N', title='Measurement'), tooltip=['year:O', 'value:Q', 'measure:N', 'location:N']).add_params(location_selector, measurement_selector).transform_filter(location_selector & measurement_selector).properties(title="Year Based Time Series Chart of Selected Measurements by Location", width=800, height=400)
time_series_chart

##### Key Insights

Based on the selected measures ('Total dissolved salts,' 'Bicarbonates,' 'Oxygen saturation,' 'Total hardness,' 'Zinc,' 'Calcium,' and 'Iron') and the use of a dropdown for measures and radio buttons for locations, we can uncover some interesting insights over the years:

Dynamic Exploration: The dropdown allows users to switch between measures, highlighting variations and trends for each chemical component.

Location-Based Insights: The radio buttons provide a focused view of how these measures behave across different locations, revealing location-specific trends and patterns.

Temporal Trends: The visual facilitates an understanding of how the selected measures evolve over the years, helping identify spikes, consistent values, or unique behaviors for specific measures and locations.



In [ ]:
# Identify the top observed chemical contaminations (top 20 by frequency)
top_chemicals = df['measure'].value_counts().head(20).index.tolist()

top_df = df[df['measure'].isin(top_chemicals)]

line_chart_year = alt.Chart(top_df).mark_line().encode(x=alt.X('year:O', title='Year'), y=alt.Y('count():Q', title='Frequency'), color=alt.Color('measure:N', title='Chemical'), tooltip=['year:O', 'measure:N', 'count():Q']).properties(title='Yearly Frequency of Observations for Top Chemicals', width=800, height=400)
line_chart_year


##### Key Insights

The line chart illustrates the trends over time for the top 20 chemical components. Notably, after 2004, there is a significant divergence among the lines, indicating greater variability in the chemical components. Additionally, a sharp rise in water temperature is observed in 2015, highlighting a potential anomaly or critical event during that year.

In [ ]:
# Group and aggregate data to get the sum of values by location and measure
aggregated_data = df.groupby(['location', 'measure']).agg({'value': 'sum'}).reset_index()

stacked_bar = alt.Chart(aggregated_data).mark_bar().encode(x=alt.X('sum(value):Q', title='Sum of Value'), y=alt.Y('location:N', title='Location', sort='-x'), color=alt.Color('measure:N', title='Measure', scale=alt.Scale(scheme='category10')), tooltip=['location:N', 'measure:N', 'sum(value):Q']).properties(title="Top Contaminations – Waterways", width=800, height=400)
stacked_bar


##### Key Insights

The stacked bar chart visualizes the top contaminated waterways, with chemical components represented by different colors. From the chart, it is evident that Chai is the most contaminated waterway, followed by Boonsri and Kannika, emphasizing the severity of contamination in these locations.

Additionally, Tansanee, Decha, and Achara are identified as the three locations with the lowest levels of contamination, indicating relatively better water quality in these areas.

In [ ]:
# Filter data for water temperature
water_temp_data = df[df['measure'] == 'Water temperature']

avg_temp = water_temp_data.groupby(['location', 'year', 'month']).agg({'value': 'mean'}).reset_index()

bar_chart_by_year = alt.Chart(avg_temp).mark_bar().encode(x=alt.X('year:O', title='Year'), y=alt.Y('mean(value):Q', title='Avg Water Temperature'), color=alt.Color('location:N', title='Location'), tooltip=[alt.Tooltip('year:O', title='Year'), alt.Tooltip('location:N', title='Location'), alt.Tooltip('mean(value):Q', title='Avg Temp', format='.2f')]).properties(title="Average Water Temperature by Year for All Locations", width=800, height=400)

bar_chart_by_month = alt.Chart(avg_temp).mark_bar().encode(x=alt.X('month:O', title='Month'), y=alt.Y('mean(value):Q', title='Avg Water Temperature'), color=alt.Color('location:N', title='Location'), tooltip=[alt.Tooltip('month:O', title='Month'), alt.Tooltip('location:N', title='Location'), alt.Tooltip('mean(value):Q', title='Avg Temp', format='.2f')]).properties(title="Average Water Temperature by Month for All Locations", width=800, height=400)

combined_chart = bar_chart_by_year & bar_chart_by_month
combined_chart


##### Key Insights

Year-Based:
Water temperature begins to rise significantly from 2009 and remains consistently high through 2016, indicating a long-term upward trend during this period.

Month-Based:
Water temperature follows a bell-curve pattern, starting to rise in April, peaking in July, and then showing a downward trend from August onwards. This creates a near-perfect bell curve, reflecting seasonal variations.


In [ ]:
# Filter data for "Dissolved Oxygen" and "Water Temperature"
filtered_data_oxy = df[df['measure'] == 'Dissolved oxygen'].copy()
filtered_data_temp = df[df['measure'] == 'Water temperature'].copy()

merged_data = pd.merge(filtered_data_oxy[['sample date', 'location', 'value']], filtered_data_temp[['sample date', 'location', 'value']], on=['sample date', 'location'], suffixes=('_oxy', '_temp'))

# Create the scatter plot
scatter = alt.Chart(merged_data).mark_circle(color='orange', opacity=0.6).encode(x=alt.X('value_temp:Q', title='Water Temperature (°C)'), y=alt.Y('value_oxy:Q', title='Dissolved Oxygen (mg/L)'), tooltip=[alt.Tooltip('value_temp:Q', title='Water Temperature (°C)', format='.2f'), alt.Tooltip('value_oxy:Q', title='Dissolved Oxygen (mg/L)', format='.2f'), alt.Tooltip('location:N', title='Location'), alt.Tooltip('sample date:T', title='Sample Date')]).properties(title="Correlation between Dissolved Oxygen and Water Temperature", height=400, width=500)

trend_line = scatter.transform_regression('value_temp', 'value_oxy', method="linear").mark_line(color='gray')

combined_chart = scatter + trend_line
combined_chart

##### Key Insights

Above chart highlights a strong negative correlation between water temperature and dissolved oxygen levels.

In [ ]:
# Define the outlier detection function
def findOutliers(group):
    for measure in group['measure'].unique():
        measure_data = group[group['measure'] == measure]
        Q1 = measure_data['value'].quantile(0.25)
        Q3 = measure_data['value'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - (1.5 * IQR)
        upper_bound = Q3 + (1.5 * IQR)
        
        outlier_condition = (measure_data['value'] < lower_bound) | (measure_data['value'] > upper_bound)
        group.loc[measure_data.index, 'Outlier_Flag'] = outlier_condition
    return group

df['Outlier_Flag'] = False
outlier_df = df.groupby('location').apply(findOutliers)
outlier_df = outlier_df.drop(columns=['location']).reset_index()

measures = ['Nickel', 'Lead', 'Chromium', 'Copper', 'Manganese', 'Iron']
filtered_data = outlier_df[outlier_df['measure'].isin(measures)]

# Create radio button for location selection
location_selection = alt.selection_point(fields=['location'], bind=alt.binding_radio(options=filtered_data['location'].unique().tolist(), name='Select Location'))

# Base scatter plot
base = alt.Chart().mark_point().encode(x=alt.X('sample date:T', title='Sample Date'), y=alt.Y('value:Q', title='Value'), color=alt.Color('Outlier_Flag:N', title='Outlier', scale=alt.Scale(domain=[True, False], range=['red', 'steelblue'])), tooltip=[alt.Tooltip('sample date:T', title='Date'), alt.Tooltip('Outlier_Flag:N', title='Outlier'), alt.Tooltip('value:Q', title='Value')]).properties(width=250, height=250).add_params(location_selection).transform_filter(location_selection)


# Generate individual scatter plots for each measure
rows = []
for i in range(0, len(measures), 3):
    row = alt.hconcat(
        *[base.transform_filter(alt.datum.measure == measure).properties(title=measure) for measure in measures[i:i+3]]
    )
    rows.append(row)

outlier_chart = alt.vconcat(*rows, data=filtered_data).resolve_scale(x='independent', y='independent').properties(title="Outliers Scatter Plot for Selected Measures")

# Display the visualization
outlier_chart

##### Key Insights

The multi-scatter chart provides better visualization of outliers for the top 6 chemical components across various waterways (locations). Waterways are selectable using radio buttons.

Locations Sakda, Kannika, Chai, and Boonsri exhibit higher concentrations of outliers, indicating more significant contamination levels for the selected chemical components in these waterways.